## Libraries

In [67]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from pathlib import Path

In [68]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [69]:
from sklearn.svm import SVR

## Load data

In [107]:
data = pd.read_csv("..//bike_sharing_day.csv")

In [108]:
data["day"] = pd.DatetimeIndex(data.dteday).day
data = data.drop("dteday", axis = 1)

In [123]:
data = pd.get_dummies(data, columns = ["season", "mnth", "weekday"], drop_first = True)

In [124]:
# Spliting target variable and independent variables
X = data.drop(['cnt'], axis = 1)
y = data['cnt']

In [125]:
# Matrix with feature transformation
# X = pd.read_csv("..//newBike_sharing_day.csv")

In [126]:
# Splitting to training and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 4)

In [127]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Optimization function

In [128]:
def opt_bas(C, epsilon, gamma):
    
    # parameters
    hyperparameters = {
        'kernel' : "rbf",
        'C' : C, 
        'epsilon' : epsilon, 
        'gamma' : gamma,
    }
    
    # fit and predict
    model = SVR(**hyperparameters).fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    # get score
    metric = mean_absolute_error(y_test, y_pred)
    
    return -metric

## Run optimization

In [129]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [132]:
# Bounded region of parameter space
pbounds = {'C': (5000, 20000), 'epsilon': (0.1, 30), 'gamma': (0.0001, 0.05)}

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=1, verbose=2, 
                                )

# keep data
log_path = Path().resolve() / "Logs" / "Boston_mae_leo_dum.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [133]:
optimizer.maximize(init_points=6, n_iter=500)